# NLP-based Competencies Extraction Service

##  Tools and Libaries
pandas: a software library written for the Python programming language for data manipulation and analysis

spacy: an open-source software library for advanced natural language processing

de_core_news_lg:  trained pipelines for German language

In [617]:
# pip install spacy
# python -m spacy download de_core_news_lg
import pandas as pd
import spacy
sp = spacy.load('de_core_news_lg')

## Dataset

### ESCO Skill Dataset

In [618]:
skills = pd.read_csv('./data/skills_de.csv')[['conceptUri','preferredLabel','description']]

In [619]:
skills

,conceptUri,preferredLabel,description
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...
1,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...
2,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ..."
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...
...,...,...,...
13886,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,berufliche Leistungsfähigkeit von Nutzern/Nutz...,"Wiederherstellen der kognitiven, sensomotorisc..."
13887,http://data.europa.eu/esco/skill/fff0b074-5a76...,Beleuchtung in Transportgeräten einbauen,Einbau von Beleuchtungselementen in Transportg...
13888,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,Verarbeitung natürlicher Sprache,"Technologien, die es IKT-Geräten ermöglichen, ..."
13889,http://data.europa.eu/esco/skill/fff5bc45-b506...,Bauarbeiten koordinieren,Koordinierung der Tätigkeiten mehrerer Bauarbe...


### DIN PAS 1045 Course Description Dataset

In [620]:
courses = pd.read_csv('./data/all_courses1.csv')

In [621]:
courses

,course_name,course_id,course_description
0,"""Schwierige"" Klienten? - Mit Patienten, Angehö...",4A873264-7ADD-DE47-3039-1FDA692E8164,- Analyse strapaziöser Gesprächsmuster\n- Schw...
1,Aktuelles Arbeitsrecht 2022,99BEDCEB-4FF3-3F8F-88B7-B30E50638F01,Kurzbeschreibung\nDas Arbeitsrecht unterliegt ...
2,Ambulante Pflege - Rechtssicher Handeln und Ha...,97FD41FD-1A91-179C-3EA5-6A23E2F436D5,- Grundlagen der straf- und zivilrechtlichen H...
3,Aufgaben des gesetzlichen Betreuers - Zur Refo...,A0CC573A-79D7-79A4-5B22-F675C4F06950,Kurzbeschreibung\nNoch im Jahr 2020 plant der ...
4,Basisqualifikation für ungelernte Pflegekräfte...,7C449EF7-12A8-82FC-0E5E-BD2885FDB93C,- Alten- und Krankenpflege\n . Körperpflege\n...
...,...,...,...
16847,5 Monate Weiterbildung: Organisation & Führung...,A1E046BE-1C4B-B977-C9B9-88D8D2860107,Die aktuell vorherrschende Situation auf dem A...
16848,Conversion und Usability Experte,A1DF8EEB-D317-70BA-508F-AFC732369860,Ziel der Maßnahme ist es den Teilnehmern eine ...
16849,Digital Transformation Management,A1DD9D98-2BA3-3A11-0A24-72FC4C79422A,Ziel der Maßnahme ist es den Teilnehmern eine ...
16850,E-Commerce Geschäftsmodelle,A1DD60F7-9B6E-ECEE-F1DF-53FED2DCCC5C,Ziel der Maßnahme ist es den Teilnehmern eine ...


## Data Pre-processing


### ESCO Skills Dataset

Because the data comes from the official documents of European Commission, the quality is relatively good, so we didn't process data cleaning but directly removed the "stop words" in the data, and then perform lemmatization. Here, the `.is_stop` and `.lemma` methods provided in spacy are used. method `.lemma` will convert punctuation other than hyphens to "--", we choose non-punctuation and non-empty terms. In the end we found that "\xa0" (on-breaking space in Latin1 (ISO 8859-1)) would not be processed by the previous steps, so here we hardcoded to remove them. Currently we only use "preferredLabel" column as the feature.

###  preferredLabel

In [622]:
# labels.to_csv("./data/skillLabels.csv")
labels = skills['preferredLabel'].copy(deep=True)

In [623]:
labels_processed = []
for label in (labels):
    label = sp(label)
    label_processed = ''
    for word in label:
        word = word.lemma_.lower()
        word = str(word)
        if word != '--' and word != '' and word != ' ':
            if word == '\xa0': continue
            if word.endswith("innen") and word != 'gewinnen': continue
            label_processed += word + ' '
    labels_processed.append(label_processed[:-1])   

In [624]:
pd.DataFrame(labels_processed, columns = ['processedLabel'])

,processedLabel
0,musikpersonal verwalten
1,strafvollzugsverfahr beaufsichtigen
2,nicht unterdrückend praktik anwenden
3,einhaltung von vorschrift von eisenbahnfahrzeu...
4,verfügbar dienst ermitteln
...,...
13886,beruflich leistungsfähigkeit von nutzer nutzer...
13887,beleuchtung in transportgerät einbauen
13888,verarbeitung natürlich sprache
13889,bauarbeit koordinieren


#### description

In [625]:
skill_descriptions = skills['description'].copy(deep=True)

In [626]:
skill_descriptions

0        Zuweisen und Verwalten der Aufgaben des Person...
1        Überwachen des Betriebs einer Justizvollzugsan...
2        Ermitteln von Repressionen in Gesellschaften, ...
3        Kontrollieren von Fahrzeugen, Komponenten und ...
4        Ermitteln der verschiedenen verfügbaren Dienst...
                               ...                        
13886    Wiederherstellen der kognitiven, sensomotorisc...
13887    Einbau von Beleuchtungselementen in Transportg...
13888    Technologien, die es IKT-Geräten ermöglichen, ...
13889    Koordinierung der Tätigkeiten mehrerer Bauarbe...
13890    Anbringen von Absturzsicherungen und Bordbrett...
Name: description, Length: 13891, dtype: object

### DIN PAS 1045 Course Description Dataset
We used "course_name" and "couse_description" columns together as the feature.

In [627]:
courses['course_info'] = courses['course_name'] + " " + courses['course_description']
courses_info = courses['course_info'].copy(deep=True)

In [638]:
courses_info_processed = []
for course_info in (courses_info):
    course_info = sp(course_info)
    course_info_processed = ''
    for word in course_info:
        if not word.is_stop:
            word = word.lemma_.lower()
            if word != '--' and word != "" and word != " ":
                if "/-" in word: word = word.split("/-")[0]
                course_info_processed += word + ' '
    courses_info_processed.append(course_info_processed)  

In [639]:
pd.DataFrame(courses_info_processed, columns = ['coursesInfoProcessed'])

,coursesInfoProcessed
0,schwierig klient patient angehörige kollege cl...
1,aktuelles arbeitsrecht 2022 kurzbeschreibung \...
2,ambulant pflege rechtssicher handeln haftungsr...
3,aufgabe gesetzlich betreuer reform betreuungsr...
4,basisqualifikation ungelernt pflegekräft zerti...
...,...
16847,5 monat weiterbildung organisation & führung l...
16848,conversion usability experte ziel maßnahme tei...
16849,digital transformation management ziel maßnahm...
16850,e-commerce geschäftsmodell ziel maßnahme teiln...


## NLP Algorithms

### Ontology-based Entity Recognition

`termStore:  {controlled vocabulary (vocabularies in label): URI}`

In [632]:
termStore = {}
URI = 0
for label_processed in labels_processed:
    label_processed = sp(label_processed)
    for word in label_processed:
        word = str(word)
        if word not in termStore:
            termStore[word] = URI
            URI += 1

In [633]:
pd.DataFrame(termStore.items(), columns = ['controlledVocabulary', 'URI'])

,controlledVocabulary,URI
0,musikpersonal,0
1,verwalten,1
2,strafvollzugsverfahr,2
3,beaufsichtigen,3
4,nicht,4
...,...,...
12194,scala,12194
12195,bodentragfähigkeit,12195
12196,bibliotheksartikel,12196
12197,absturzsicherung,12197


`sequenceStore: {URIs : (index, sequence consisted of controlled vocabularies (label))}`

In [634]:
sequenceStore = {}
for i,label_processed in enumerate(labels_processed):
    URIs = []
    label_processed = sp(label_processed)
    for word in label_processed:
        URIs.append(termStore[str(word)])
    sequenceStore[tuple(URIs)] = (i,str(label_processed))

In [654]:
pd.DataFrame(sequenceStore.items(), columns = ['URIs', '(index, label)']).to_csv('./data/sequenceStore.csv')

The algorithm scans the tokenized courses information from the beginning until a word contained in the `termStore` is reached. Starting from this word a lookahead is performed searching for the longest sequence of words, which are contained in the `termStore`. As soon as a subsequent term is not included in the `termStore`, the lookahead process terminates and delivers the all sequence still contained in the `termStore`. and which form a sequence contained in the `sequenceStore` by using URIs. 

In [694]:
def get_relations(index_start, index_end):
    URIs_candidates = []
    word_candidates = []
    relations = []
    courses_info_processed_subset = courses_info_processed[index_start:index_end]
    for i, course_info_processed in enumerate(courses_info_processed_subset):
        index_course = index_start + i
        for word in sp(course_info_processed):
            word = str(word)
            if word in termStore:
                word_candidates.append(word)
                URIs_candidates.append(termStore[word])
            else:
                if URIs_candidates != []:
                    URIs_candidates, relations = checkCandidates(URIs_candidates, index_course, relations)
                word_candidates = []
    URIs_candidates, relations = checkCandidates(URIs_candidates, index_course, relations)
    for relation in relations:
        print(courses['course_name'][relation[0]])
        print(' ---> ' + skills['preferredLabel'][relation[1]] + '   ' + skills['conceptUri'][relation[1]])
        print()
def checkCandidates(URIs_candidates, index_course, relations):
    n = len(URIs_candidates)
    for i in range(n):
        for j in range(i+1, n+1):
            URIs = tuple(URIs_candidates[i:j])
            if URIs in sequenceStore:
                index_label = sequenceStore[URIs][0]
                if (index_course, index_label) not in relations:
                    relations.append((index_course, index_label))
    URIs_candidates = []
    return URIs_candidates, relations

In [695]:
get_relations(300,400)

Englisch - Grundkurs (Seminar zur Grundversorgung nach dem BbgWBG)
 ---> Englisch   http://data.europa.eu/esco/skill/6d3edede-8951-4621-a835-e04323300fa0

Finanzbuchführung EDV-unterstützt - Anwendung kaufmännischer Software - Modul 2 kaufmännischer Bereich
 ---> Jahresabschluss   http://data.europa.eu/esco/skill/4a460161-eaea-4fe2-bd0e-164ccc635be1

Finanzbuchführung Jahresabschluss / Anlagenbuchhaltung - EDV-unterstützt - Modul 11 kaufmännischer Bereich
 ---> Jahresabschluss   http://data.europa.eu/esco/skill/4a460161-eaea-4fe2-bd0e-164ccc635be1

Fräsen konventionell - Modul 13
 ---> planen   http://data.europa.eu/esco/skill/e49f4158-9d4c-425d-bf32-dfe89b19840a

Geprüfter Industriemeister/ Geprüfte Industriemeisterin Fachrichtung Metall
 ---> Kommunikation   http://data.europa.eu/esco/skill/15d76317-c71a-4fa2-aadc-2ecc34e627b7

Gesunde Ernährung für die Familie (Seminar zur Grundversorgung nach dem BbgWBG)
 ---> Ernährung   http://data.europa.eu/esco/skill/13d96098-2ff0-46b8-b326-b7c

# Testing

In [357]:
for sequence in courses_info_processed[1].split("\n"):
    if (len(sequence) != 0): print(sequence)

aktuelles arbeitsrecht 2022kurzbeschreibung 
 arbeitsrecht unterliegen ständig wandel änderung gesetzgebung folgend rechtsprechung seminar erhalten notwendig 
 kenntnis vorschrift beziehen sozial einrichtung rechtssicher handeln unnötig gerichtlich auseinandersetzung vermeiden 
 beschäftigtendatenschutz eudsgvo bdsg 
 urlaubsrecht urlaubsplanung verfall anspruch abgeltung urlaub 
 qualifizierungsvereinbarunge bindungsklausel 
 befristung arbeitsverträgen sachgrund 
 inhaltliche gestaltungsmöglichkeit arbeitsverträg 
 arbeitszeitrecht bereitschaftsdienste über mehrstunden 
 arbeitsrechtlich sanktion ermahnung abmahnung kündigung 
 bedeutung eingliederungsmanagement personenbedingt kündigung krankheit 
 übersicht aktuell rechtsprechung arbeitsgericht 
 grund aktualität kurzfristig thema aufnehmen      


In [327]:
d1 = sp("aufbau organisation im eisenbahnbetrieb")
d2 = sp("risikomanagement eisenbahnbetrieb durchführen")
d3 = sp("management finanzrisik durchführen")
d1.similarity(d2)

0.30732991357106343

In [448]:
skill_labels[1029]

'Flughafenplanung'

In [378]:
course_descriptions[4354]

'Der Fluss von elektrischer Ladung durch Elektronen oder Ionen in einem Medium wie einem Elektrolyten oder einem Plasma.'

In [ ]:
# code from paper discard
# def annotate (text):
#     tt = sp(text)
#     s = p = 0
#     at = ''
#     a = []
#     for token in tt:
#         p += 1
#         if s > 1:
#             s -= 1
#             continue
#         elif token.lemma_ not in termStore:
#             at = at + ' ' + token
#         else: 
#             phrase,l = lookahead(tt[p+1:], [token], 1)
#             if phrase != []:
#                 URI = sequenceStore.get(phrase)
#                 if URI != None:
#                     a = a.append((p, phrase, URI))
#                     at += ' ' + phrase
#                     continue
#             at += ' ' + token
#         return a, at

# def lookahead(tt, fp, n):
#     if len(tt) == 0: return fp, n 
#     termFound = tt[0].lemma_.lower() in termStore                                         
#     phraseFound = tuple(fp) in sequenceStore
#     if termFound or phraseFound:                                   
#         fp.append(tt[0])
#         ph, l = lookahead(tt[1:], fp, n+1)
#         print(fp, ph)
#         if tuple(ph) in sequenceStore: return ph, l
#         elif phraseFound: return fp, n
#     return ([],n)